## The number of people owed a prevention duty 

Now to compare the number of people owed a prevention duty in the final quarter on 2024 - to the number of social housing units proposed in 2024. 
The data on the number of people owed a prevention duty is available on gov.uk: [https://www.gov.uk/government/statistical-data-sets/live-tables-on-homelessness](https://www.gov.uk/government/statistical-data-sets/live-tables-on-homelessness). 

In [1]:
import pandas as pd
import json
import time
import random
import requests
import urllib3

from elasticsearch import Elasticsearch

import geopandas as gpd
from geopy.geocoders import Nominatim

import seaborn as sns
from matplotlib.colors import to_hex

import plotly.graph_objects as go
import plotly.io as pio
from shapely.geometry import Polygon, Point
import plotly.express as px

import sys
# Import custom functions which help process and format the planning data 
sys.path.append('functions')
import preprocessing_fncs as ppf
import elastic_search_fncs as esf

In [2]:
df_pd = pd.read_excel(r'data/Detailed_LA_202412.xlsx', sheet_name='A1')
df_social_housing_borough = pd.read_csv(r'data/formatted_data/social_housing_borough.csv')

In [3]:
df_pd.columns = ['LAD_id',
 'Borough',
 'nan',
 'nan',
 'Total number of households assessed1,2',
 'nan',
 'Total households assessed as owed a duty',
 'Threatened with homelessness - Prevention duty owed',
 'Of which: due to service of valid Section 21 Notice',
 'Homeless - Relief duty owed',
 'nan',
 'Not threatened with homelessness within 56 days - no duty owed',
 'Withdrew application  before assessment - no duty owed',
 'Not eligible / no longer eligible - no duty owed',
 'nan',
 'nan',
 'nan',
 'nan']

In [4]:
df_pd.head(12)

,LAD_id,Borough,nan,nan,"Total number of households assessed1,2",nan,Total households assessed as owed a duty,Threatened with homelessness - Prevention duty owed,Of which: due to service of valid Section 21 Notice,Homeless - Relief duty owed,nan,Not threatened with homelessness within 56 days - no duty owed,Withdrew application before assessment - no duty owed,Not eligible / no longer eligible - no duty owed,nan,nan,nan,nan
0,NaN,NaN,NaN,NaN,"Total number of households assessed1,2",NaN,Total households assessed as owed a duty,Threatened with homelessness - Prevention duty...,Of which: due to service of valid Section...,Homeless - Relief duty owed,NaN,Not threatened with homelessness within 56 day...,Withdrew application before assessment - no d...,Not eligible / no longer eligible - no duty owed,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,Grand Total,NaN,NaN,Threatened with homelessness – Prevention Duty...,Threatened with homelessness due to service of...,Already homeless – Relief Duty owed (include a...,NaN,Not threatened with homelessness within 56 days,Withdrew application before assessment - no d...,Not eligible / no longer eligible - no duty owed,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Number of households,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,"Total initial assessments1,2,6",NaN,Assessed as owed a duty,NaN,NaN,NaN,NaN,Not homeless nor threatened with homelessness ...,Withdrew application before assessment - no d...,Not eligible / no longer eligible - no duty owed,NaN,Number of households\n in area5 (000s),Households assessed as threatened with homeles...,Households assessed as homeless\nper (000s)
4,NaN,NaN,NaN,NaN,NaN,NaN,Total owed a prevention or relief duty,Threatened with homelessness within 56 days - ...,Of which:,Homeless - \nRelief duty owed4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,due to service of valid Section 21 Notice3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,E92000001,ENGLAND,NaN,NaN,83800,NaN,76820,34010,5820,42810,NaN,3310,3070,600,NaN,24209.029,1.404817,1.768463
7,E12000007,London,NaN,NaN,18860,NaN,17250,6890,1240,10360,NaN,570,920,120,NaN,3659.46,1.88189,2.831091
8,-,Rest of England,NaN,NaN,64930,NaN,59580,27120,4590,32450,NaN,2740,2140,480,NaN,20549.569,1.31986,1.57923
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df_pd = df_pd[6:]

In [6]:
df_pd.drop(columns=['nan'], inplace=True)

In [7]:
df_pd = ppf.format_df(df_pd, borough_col_name='Borough')

In [30]:
boroughs = ['Barking and Dagenham', 'Barnet', 'Camden', 'Ealing', 'Hackney',
       'Havering', 'Hounslow', 'Islington', 'Lambeth', 'Redbridge',
       'Greenwich', 'Merton', 'Bromley', 'Southwark', 'Waltham Forest',
       'Wandsworth', 'Brent', 'Croydon', 'Harrow', 'Newham',
       'Tower Hamlets', 'Bexley', 'Enfield', 'Hammersmith and Fulham',
       'Haringey', 'Hillingdon', 'Kingston upon Thames', 'Lewisham', 'Richmond upon Thames',
       'Sutton', 'Westminster', 'City of London', 'Kensington and Chelsea']

In [32]:
df_pd = df_pd[df_pd['Borough'].isin(boroughs)]

In [37]:
df_pd = df_pd[['Borough', 'Total number of households assessed1,2']]

In [33]:
df_social_housing_borough = ppf.format_df(df_social_housing_borough, borough_col_name='Planning authority')

In [51]:
# define inner and outer boroughs
inner_boroughs = df_social_housing_borough[df_social_housing_borough['Region'] == 'Inner']['Planning authority'].unique()
outer_boroughs = df_social_housing_borough[df_social_housing_borough['Region'] == 'Outer']['Planning authority'].unique()

# dicrete color map
outer_palette = sns.color_palette("RdPu", n_colors=len(outer_boroughs))
inner_palette = sns.color_palette("PuBuGn", n_colors=len(inner_boroughs))

outer_colors_hex = [to_hex(c) for c in outer_palette]
inner_colors_hex = [to_hex(c) for c in inner_palette]

# Assign colors to boroughs
color_map = {}

for area, color in zip(inner_boroughs, inner_colors_hex):
    color_map[area] = color

for area, color in zip(outer_boroughs, outer_colors_hex):
    color_map[area] = color

In [52]:
df_social_housing_borough = df_social_housing_borough[df_social_housing_borough['Planning authority'].isin(boroughs)]

In [53]:
import plotly.graph_objects as go

# Calculate total units by borough
borough_totals = (
    df_social_housing_borough
    .groupby('Planning authority')['Number of proposed social housing units']
    .sum()
    .sort_values(ascending=False)
)

# Get region info for hover labels
borough_regions = (
    df_social_housing_borough
    .drop_duplicates('Planning authority')
    .set_index('Planning authority')['Region']
)

# Determine height: 25 pixels per borough, with a minimum height
bar_height = 25
min_height = 400
height = max(min_height, bar_height * len(borough_totals))

# Create bar chart
fig = go.Figure()

fig.add_trace(go.Bar(
    x=borough_totals.values,
    y=borough_totals.index,
    orientation='h',
    marker=dict(
        color=[color_map.get(b, '#999999') for b in borough_totals.index]
    ),
    hovertemplate=(
        '<b>%{y}</b><br>'
        'Region: %{customdata[0]}<br>'
        'Total Proposed Units: %{x}<br>'
        '<extra></extra>'
    ),
    customdata=[[borough_regions.get(b)] for b in borough_totals.index]
))

# Update layout
fig.update_layout(
    title='Fig 6. Total Proposed Social Housing Units by Borough (2015-2025)',
    xaxis_title='Proposed Social Housing Units',
    yaxis_title='Borough',
    template='plotly_white',
    autosize=False,
    height=height,
    yaxis=dict(autorange="reversed")  # Largest at the top
)

fig.show()


In [57]:
# Save the fig a a json
pio.write_json(fig, "outputs/council_proposed_london_stacked_bar.json")

In [58]:
# Filter 2024 data for proposed social housing units
df_2024 = df_social_housing_borough[df_social_housing_borough['Year'] == 2024]

# Merge with prevention duty data to align boroughs
df_compare = pd.merge(
    df_2024[['Planning authority', 'Number of proposed social housing units']],
    df_pd[['Borough', 'Total number of households assessed1,2']],
    left_on='Planning authority',
    right_on='Borough',
    how='inner'
)

# Convert assessed households to numeric
df_compare['Total number of households assessed1,2'] = pd.to_numeric(
    df_compare['Total number of households assessed1,2'], errors='coerce'
)

# Sort by borough for consistent plotting
df_compare.sort_values('Planning authority', inplace=True)


# Sort by number of proposed units, descending
df_compare_sorted = df_compare.sort_values(
    'Number of proposed social housing units',
    ascending=False
)

# Plot
fig_compare = go.Figure()

fig_compare.add_trace(go.Bar(
    y=df_compare_sorted['Planning authority'],
    x=df_compare_sorted['Number of proposed social housing units'],
    name='Proposed Social Housing Units (2024)',
    marker_color='#047495',
    orientation='h'
))

fig_compare.add_trace(go.Bar(
    y=df_compare_sorted['Planning authority'],
    x=df_compare_sorted['Total number of households assessed1,2'],
    name='Households Owed a Prevention Duty (2024 Q4)',
    marker_color='#c20078',
    orientation='h'
))

fig_compare.update_layout(
    barmode='group',
    title='Fig. 7: Proposed Social Housing Units vs Households Owed a Prevention Duty',
    xaxis_title='Count',
    yaxis_title='Borough',
    template='plotly_white',
    height=1000,
    yaxis=dict(autorange="reversed")  # Largest at the top
)

fig_compare.show()




In [59]:
# Save the fig a a json
pio.write_json(fig_compare, "outputs/council_proposed_vs_prevention_stacked_bar.json")